# **Predicting Parkinson's using EMD features and XGBoost**


In [1]:
!pip install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.4.2 xgboost==2.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 102.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 122.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 115.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 17.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 33.3 MB/s eta 0:00:00


## Import required libraries

In [2]:
from tqdm import tqdm # Imports `tqdm` to create a progress bad in Python

import pandas as pd # Imports the pandas library for tabular data
import numpy as np # Imports the numpy library for mathematical operations and linear algebra

import sklearn.metrics # Import metrics from sklearn
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, GridSearchCV # Import functions for cross validation

import xgboost as xgb # Import XGBoost for gradient boosting

## Import datasets

The following code block imports, processes, and concatenates the datasets. The datasets come from Alice Rueda's [GitHub page](https://github.com/alicerueda/DDK-EMD-Feature-Extraction) and were used in the following paper:
> Rueda, Alice, et al. "Empirical mode decomposition articulation feature extraction on Parkinson’s diadochokinesia." *Computer Speech & Language* 72 (2022): 101322.

In the construction of this dataset Empirical Mode Decomposition's (EMD) dyadic filterbank characteristics were used to extract Diadochokinesia (DDK) features. The EMD analysis on DDK looks at the spectrum characteristics of Intrinsic Mode Functions (IMF) and the handling of mode mixing conditions. The dataset includes DDK recordings of Healthy Control (`HC`) subjects and patients with Parkinson’s disease (`PD`). The objective of the paper was to show that the inclusion of certain features along with certain preprocessing strategies results in superior Parkinson's disease prediction than the exclusion of these features or their preprocessing strategies. For details, please refer to the paper.


In [3]:
# Construct dataframe for analysis

# Triad 10 file locations
HCaddr = ['https://github.com/alicerueda/DDK-EMD-Feature-Extraction/raw/master/EMD%20Features/EMDDeltaEMD/EMDHCTriadFC10Per.csv']
PDaddr = ['https://github.com/alicerueda/DDK-EMD-Feature-Extraction/raw/master/EMD%20Features/EMDDeltaEMD/EMDPDTriadFC10Per.csv']


In [5]:
# Import the HC file as a Pandas DataFrame
dfHCOriginal = pd.read_csv(HCaddr[0])

In [7]:
# Drop rows with missing data
dfHCOriginal.dropna()

,subject,segmentNum,numIMFMean,OBW1Mean,OBW2Mean,OBW3Mean,OBW4Mean,OBW5Mean,OBW6Mean,OBW7Mean,...,deltaFCenter1Kurt,deltaFCenter2Kurt,deltaFCenter3Kurt,deltaFCenter4Kurt,deltaFCenter5Kurt,deltaFCenter6Kurt,deltaFCenter7Kurt,deltaFCenter8Kurt,deltaAmpKurt,deltaDurKurt
0,1,7,17.857143,5204.862591,2866.093206,1509.694567,996.851513,638.425536,375.635234,300.511143,...,0.135277,-0.441106,0.048946,0.262891,-0.392146,0.617342,0.425867,0.010991,-0.635646,0.899006
1,3,14,18.000000,5650.135386,2411.363509,1424.045030,999.005992,690.549767,473.509724,332.738783,...,-0.470161,6.166162,-0.348186,-0.194057,-0.288391,1.057262,-0.197758,1.244823,0.677967,2.211619
2,4,8,18.375000,2821.118524,1726.278599,1151.101395,845.980157,580.557635,378.807473,257.382885,...,2.632724,0.476044,-0.247550,0.972455,0.272393,-0.799764,0.240755,-0.753770,0.690796,-1.302787
3,5,5,18.400000,5834.389918,2996.736955,1512.771899,921.499627,650.367109,435.002445,308.511409,...,-1.247197,-0.588006,-1.112773,-1.692116,-0.500388,-1.823599,-1.638865,-1.152393,-1.331687,-1.178321
4,6,11,17.818182,7011.367459,3144.921249,1916.343683,1139.951071,799.872252,529.310212,296.321417,...,-0.241626,-0.601686,-0.831340,-1.506145,0.589599,-0.555472,-1.236597,-0.067719,-0.616519,0.809568
5,7,6,16.500000,5137.451498,2657.261005,1677.718841,1147.215655,766.161432,496.949070,290.027158,...,-0.404576,-1.310518,0.142033,-0.110939,0.371341,-0.838500,-0.746416,-0.386109,-1.688892,0.983417
6,8,8,16.625000,4750.567062,1647.564072,1089.562467,759.085380,550.114383,418.761489,275.250421,...,-0.132787,-0.732157,2.775011,1.831231,1.427751,-1.112000,-1.103272,-1.220001,-1.011334,0.647161
7,10,14,18.500000,6876.896502,2880.698602,1653.986403,1100.094315,795.280578,520.186439,317.948641,...,-1.222810,-0.974119,-0.315764,0.307904,-0.225533,-0.852538,-0.520996,-0.693960,-0.524622,-1.403370
8,11,8,17.625000,6899.395777,2480.995244,1375.981044,793.947250,513.551673,341.085618,242.201075,...,-1.249766,-1.293459,-1.324131,-1.461301,-1.157726,-0.656829,0.747288,0.603278,-0.116104,0.462076
9,12,6,17.500000,4905.755888,2614.675797,1493.990274,1034.969962,723.655013,512.479858,332.011482,...,-0.176176,-1.570545,-0.867944,0.084485,-0.655197,-1.389820,-1.005582,-0.506858,-1.202627,0.510023


In [8]:
# Print the dimensions of the HC DataFrame
print('HC matrix dim: ', dfHCOriginal.shape)

HC matrix dim:  (50, 434)


In [9]:
# Import the HCPD file as a Pandas DataFrame
dfPDOriginal = pd.read_csv(PDaddr[0])

In [11]:
# Drop rows with missing data
dfPDOriginal.dropna()

,subject,segmentNum,numIMFMean,OBW1Mean,OBW2Mean,OBW3Mean,OBW4Mean,OBW5Mean,OBW6Mean,OBW7Mean,...,deltaFCenter1Kurt,deltaFCenter2Kurt,deltaFCenter3Kurt,deltaFCenter4Kurt,deltaFCenter5Kurt,deltaFCenter6Kurt,deltaFCenter7Kurt,deltaFCenter8Kurt,deltaAmpKurt,deltaDurKurt
0,1,3,18.333333,6459.855687,3607.634698,1596.621205,1018.822077,706.650428,540.822932,334.855158,...,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000
1,2,3,18.666667,5136.504378,2644.775542,1656.241725,1115.558041,689.231590,453.374635,305.409360,...,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000,-1.500000
2,3,2,20.000000,6274.921153,3788.846259,1987.403591,1157.345254,589.830389,427.816995,290.513102,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
3,5,6,18.166667,5103.797190,1646.166478,906.052268,575.968463,325.746548,214.163639,178.957028,...,0.033993,-0.346819,-1.500496,-1.466391,-1.793625,-1.617064,-0.871589,-1.108624,-1.161323,-0.469656
4,6,8,18.375000,6217.681607,2585.582285,1429.818184,983.518831,645.517799,436.476147,300.711959,...,-0.668467,0.145985,-1.172224,-0.351069,1.790834,-1.340031,-0.686116,0.382600,-0.834123,-1.399080
5,7,10,18.700000,6891.538538,3038.206970,1621.950853,1057.156618,711.403791,451.174315,304.063248,...,-0.493360,-1.047576,-0.628793,0.250853,0.471669,0.464322,-1.408225,-0.739158,-0.915768,-0.641562
6,8,18,18.555556,4653.706403,2389.387434,1379.894393,919.744332,611.032526,413.352924,246.795510,...,-0.307147,0.342193,-0.417608,-0.945703,-1.317493,-1.445031,-0.793915,-0.601410,1.072389,0.230495
7,9,6,18.666667,6411.388973,3154.402700,1697.846856,1183.222640,767.353105,497.727820,327.607524,...,-0.774912,-0.039873,-0.358632,-0.381001,-0.425689,-0.768539,0.197491,-0.290710,-0.176715,-1.090540
8,10,4,18.750000,5078.350346,2659.612082,1386.500660,825.574640,562.640349,381.632603,252.098258,...,-1.062902,-1.000112,-1.025063,-0.998216,-0.971847,-1.012675,-0.978351,-0.792020,-1.159739,-0.737128
9,11,14,17.214286,4033.463708,2203.664128,1247.902231,867.711851,565.853791,356.887396,275.162423,...,-0.889628,-0.636971,0.909055,-1.044147,0.223474,-0.775785,-0.546906,-1.075846,-1.271137,2.046683


In [12]:
# Print the dimensions of the PD DataFrame
print('PD matrix dim: ',dfPDOriginal.shape)


PD matrix dim:  (50, 434)


In [13]:
# A utility function that removes rows that contain 0 for all std features
def removeZeroData(dftmp):
  colnames = dftmp.columns
  stdcol = [header for header in colnames if 'Std' in header]
  stdtotal = dftmp[stdcol].sum(axis=1)
  dropIdx = [i for i,std in enumerate(stdtotal) if std==0]
  #print(stdtotal)
  print('Removing ', len(dropIdx), ' rows')
  for i in dropIdx:
    dftmp = dftmp.drop(dftmp.index[i])
  return dftmp

In [14]:
# Remove rows that contain 0 for all std features from the PD DataFrame
dfPDOriginal = removeZeroData(dfPDOriginal)

Removing  0  rows


In [15]:
# Get the shape of the PD DataFrame
pdrow, pdcol = dfPDOriginal.shape

In [16]:
# Print the shape of the PD DataFrame
print(dfPDOriginal.shape)

(50, 434)


In [17]:
# Remove rows that contain 0 for all std features from the HC DataFrame
dfHCOriginal = removeZeroData(dfHCOriginal)
# Get the shape of the HC DataFrame
hcrow, hccol = dfHCOriginal.shape
# Print the shape of the HC DataFrame
print(dfHCOriginal.shape)

Removing  0  rows
(50, 434)


In [ ]:
# Combine the HC and PD DataFrames
dfHCPD = pd.concat([dfHCOriginal,dfPDOriginal], axis=0)
# Create labels. Assign 1 to HC rows and zeroes to PD rows
hclabels = np.ones(hcrow, dtype=int)
pdlabels = np.zeros(pdrow, dtype=int)

In [ ]:
# Append the PD labels to the HC labels.
#   Note that this is consistent with how the HC and PD DataFrames were combined:
#   there, the HC and PD DataFrames were concatenated in that order
Labels = np.reshape(np.append(hclabels, pdlabels), (-1,1)) # Label for HC=1 and PD=0
# Print the shapes of the HC and PD labels
print(hcrow,pdrow)
# Print the shape of the concatenated HC and PD label object
print('Label size', Labels.shape)

In [24]:
# Get the column names
colnames = dfHCPD.columns
# Get the features from the concatenated HC PD DataFrame as a Numpy array
rescaledfeat = np.array(dfHCPD.iloc[:,1:].values)

In [25]:
# Print the column names
print('Features: ', colnames)
# Print the dimensions of the HC PD DataFrame
print('dfHCPD matrix dim: ', dfHCPD.shape)
# Print the dimensions of the feature array (one less than the HC PD DataFrame)
print('size of HCPD features: ', rescaledfeat.shape)

Features:  Index(['subject', 'segmentNum', 'numIMFMean', 'OBW1Mean', 'OBW2Mean',
       'OBW3Mean', 'OBW4Mean', 'OBW5Mean', 'OBW6Mean', 'OBW7Mean',
       ...
       'deltaFCenter1Kurt', 'deltaFCenter2Kurt', 'deltaFCenter3Kurt',
       'deltaFCenter4Kurt', 'deltaFCenter5Kurt', 'deltaFCenter6Kurt',
       'deltaFCenter7Kurt', 'deltaFCenter8Kurt', 'deltaAmpKurt',
       'deltaDurKurt'],
      dtype='object', length=434)
dfHCPD matrix dim:  (100, 434)
size of HCPD features:  (100, 433)


# Create the cross validator for the hyperparameter search

The following line creates a cross validator with 4 repeated stratified K-Folds, each with 5 splits. These folds are going to be used to find optimal hyperparameters.

Stratified K-Folds differ from K-Folds by ensuring that the proportion of classes in each training and testing set is roughly the same as in the entire training set.

Moreover, 4 repeated stratified K-Fold sets, each of 5 folds, are created, for a total of 20 folds. This is done to increase the stability of the results: with just one repeat there would have been a higher risk that models created using the best performing hyperparameters would perform best by random chance alone. By repeating the process several times, the probability that the optimal hyperparameters perform well by chance alone is drastically decreased.

Finally, it should be noted that the hyperparameter search process takes up a lot of time. In an ideal world, one would select a very large number of splits and a very large number of repeats. However, because `n_splits*n_repeats` number of models must be trained at validated on the validation split, a compromise between cross-validation robustness and hyperparameter search time must be reached. We felt that 20 models for each set of hyperparameters was sufficient to obtain reasonable results, though given more time, we would have liked to increase the number of repeats to at least 10.


In [26]:
# Create the folds for the grid search cross validation
kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=4, random_state=42)

# Load the classifier

The following line loads the classifier. In this case, we will use [XGBoost](https://github.com/dmlc/xgboost)'s gradient boosting classifier, a very popular and respected classifier.


In [27]:
# Load the classifier. In this case, we will use XGBoost's gradient boost classifier
cls = xgb.XGBClassifier()

# Create the hyperparameter search space

Here we will define the hyperparameter search space for our classifier. Note that, if you were to choose a classifier that is different than XGBoost, the parameters would be different.

Note that we will fix certain parameter values, and allow other parameters to assume more than one value by passing a list of multiple values. The grid search algorithm that follows will then find all unique combinations of these values and check which ones perform the best on the validation folds provided by our cross validator:


In [28]:
# Create a dict with the parameter values.
#   Note that for some parameters we allow multiple values. When multiple values exist,
#   the grid search algorithm will iterate through all possible combinations of these values
all_params_search = {'n_estimators': [100, 250],
              'booster': ['gbtree'],
              'lambda': [3,4],
              'alpha': [0],
              'subsample': [0.95, 1],
              'colsample_bytree': [0.015, 0.03],
              'max_depth': [8],
              'min_child_weight': [2],
              'eta': [0.07],
              'gamma': [0],
              'grow_policy': ['lossguide'],
              'device': ['cpu'],
              'verbosity': [0],
              'objective': ['binary:logistic'],
              'random_state': [42],
              'tree_method': ['hist'],
              'max_bin': [32]}



# Create the grid search cross validation object

The following line creates the grid search cross validation object. This object accepts several parameters:
- The classifier (in this case, `cls`, our XGBoost classifier)
- The hyperparameter search space dictionary (`all_params_search`)
- The cross validator that creates train-validation splits (`kf`)
- A verbosity level (set to `1` here to suppress unnecessary output)
- A scoring metric (set to `roc_auc` to use the area under the ROC curve)


In [29]:
# Create the grid search cross validation object
clf = GridSearchCV(cls, all_params_search, cv=kf, verbose=1, scoring='roc_auc')

# Perform grid search

The following cell performs grid search for all parameter combinations. Note that for each parameter combination 4 repeats of 5 folds are trained, as defined by our cross validator. Performing the grid search could take a few minutes, so please be patient.


In [30]:
# Fit the grid search cross validation object. This is the line that takes a lot of time!
clf.fit(rescaledfeat, Labels)

Fitting 20 folds for each of 16 candidates, totalling 320 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=4, n_splits=5, random_state=42),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importa...
             param_grid={'alpha': [0], 'booster': ['gbtree'],
                         'colsample_bytree': [0.015, 0.03], 'device': ['cpu'],
                         'eta': [0.07], 'gamma': [0],
                         'grow_policy': ['lossguide'], 'lambda': [3, 4],
                         'max_bin': [32], 'max_depth': [8],
                         'min_child_weight': [2], 'n_estimators': [100, 250],
                         'objective': ['binary:logistic'], 'random_state': [42],
                         'subsample': [0.95, 1], 'tree_method': ['hist'],
                         'verbosity': [0]},
             scoring='roc_auc', verbose=1)

The following cell retrieves the best scoring parameters from all possible combinations in the grid search:


In [31]:
# Get the best parameters from the grid search cross validation object.
clf.best_params_

{'alpha': 0,
 'booster': 'gbtree',
 'colsample_bytree': 0.015,
 'device': 'cpu',
 'eta': 0.07,
 'gamma': 0,
 'grow_policy': 'lossguide',
 'lambda': 3,
 'max_bin': 32,
 'max_depth': 8,
 'min_child_weight': 2,
 'n_estimators': 250,
 'objective': 'binary:logistic',
 'random_state': 42,
 'subsample': 1,
 'tree_method': 'hist',
 'verbosity': 0}

The following cel retrieves the best ROC AUC average validation score obtained for the above set of parameters:


In [32]:
# Get the best score from the grid search cross validation object.
clf.best_score_

0.8405000000000001

# Verify performance using 200 random cross validation splits

Having identified the optimal combination of parameters (`clf.best_params_`), we must now see if these parameters perform well on a numerous amount of random cross-validation splits. The following cell defines a loop which creates 200 repeats of K-Fold cross-validation splits, with each of the 200 repeats having 10 folds. For each of the 200 repeats, the average and 10-fold standard deviation of the following metrics are recorded:
- ROC AUC
- accuracy
- F1 score
- precision
- recall

Note that we are using K-Fold to split the data here, as opposed to a stratified K-Fold which would ensure that the class balance in each fold is roughly equivalent. We use the K-Fold split to be consistent with the underlying paper, and to test the robustness of our prediction model to perturbations in the class balance.

However, because we are using K-Fold as opposed to stratified K-Fold to split the data, there is a risk that we may end up with a bad split that does not contain any examples of one of the two classes in the train or test data; this can occur by random chance. Consequently, our `for` loop contains an inner `while` loop which checks for this condition and discards splits where the train and test do not contain both labels, repeating the iteration until 200 valid repeats are created.

Note that running the code below necessitates the fitting of 200*10=2000 models (one for each repeat and split) which is somewhat time consuming. The next cell should take about 5 minutes or less to run. We use the `tqdm` library on the outermost `for` loop to give us a progress bar that tracks the repeat that is currently being processed:


In [33]:
# Define some lists to store metric means
all_aucs = []
all_accuracies = []
all_f1s = []
all_precisions = []
all_recalls = []

# Define some lists to store metric standard deviations
all_aucs_std = []
all_accuracies_std = []
all_f1s_std = []
all_precisions_std = []
all_recalls_std = []

# Define the for loop
for i in tqdm(range(200)): # We have 200 repeats
    badsplit = True # initialize badplit to True
    while badsplit: # A loop that runs forever as long as there is a badsplit
        kf = KFold(n_splits=10, shuffle=True) # Create a random K-Fold cross validator with shuffle=True
        splits = list(kf.split(rescaledfeat)) # Split the data and store the splits in a list
        badsplits = [] # Initialize a list to store badsplits
        for j in splits: # A loop that populates badsplits with False or True
            if len(np.unique(Labels[j[1]])) == 2:
                badsplits.append(False) # If there are 2 labels, add False to badsplits
            else:
                badsplits.append(True) # If there aren't 2 labels, add True to badsplts
        badsplit = max(badsplits) # Set badsplit to True if any element in badsplits is True, otherwise False
    # Define some lists to store metrics for this repeat
    aucs = []
    accuracies = []
    f1s = []
    precisions = []
    recalls = []
    # Define a loop to process every one of the 10 CV splits in this repeat
    for s in splits:
        cls = xgb.XGBClassifier(**clf.best_params_) # Define classifier with best params from grid search
        cls.fit(rescaledfeat[s[0]], Labels[s[0]]) # Fit classifier to training data in the split
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1] # Get the predicted probabilities of each class
        pred_labels = np.where(preds_proba > 0.5, 1, 0) # Assign class labels from probabilities using threshold
        # Calculate metrics and append to lists for this repeat
        auc = sklearn.metrics.roc_auc_score(Labels[s[1]], pred_labels)
        aucs.append(auc)
        accuracy = sklearn.metrics.accuracy_score(Labels[s[1]], pred_labels)
        accuracies.append(accuracy)
        f1 = sklearn.metrics.f1_score(Labels[s[1]], pred_labels)
        f1s.append(f1)
        precision = sklearn.metrics.precision_score(Labels[s[1]], pred_labels)
        precisions.append(precision)
        recall = sklearn.metrics.recall_score(Labels[s[1]], pred_labels)
        recalls.append(recall)
    # Once all splits in this repeat are processed, append the means and standard deviations to global lists
    all_aucs.append(np.mean(aucs))
    all_accuracies.append(np.mean(accuracies))
    all_f1s.append(np.mean(f1s))
    all_precisions.append(np.mean(precisions))
    all_recalls.append(np.mean(recalls))
    all_aucs_std.append(np.std(aucs))
    all_accuracies_std.append(np.std(accuracies))
    all_f1s_std.append(np.std(f1s))
    all_precisions_std.append(np.std(precisions))
    all_recalls_std.append(np.std(recalls))

100%|██████████| 200/200 [03:21<00:00,  1.01s/it]


We can now check how our models performed on the 200 repeats. First off, let's identify the repeat that had the highest accuracy, and then pull out the metrics for that repeat. To do this we use the `np.argmax` function from Numpy which will provide us with the index within the accuracy list that is the highest. We can then use this index to retrieve the other metrics from their relevant lists. Note that the results will be in the form:
> `metric: mean, std`

where `mean` is the mean for that repeat across the 10 splits and `std` is the standard deviation for the repeat across the 10 splits: 


In [34]:
best_acc_idx = np.argmax(all_accuracies)
print('accuracy: ' + str(all_accuracies[best_acc_idx]) + ', ' + str(all_accuracies_std[best_acc_idx]))
print('precision: ' + str(all_precisions[best_acc_idx]) + ', ' + str(all_precisions_std[best_acc_idx]))
print('recall: ' + str(all_recalls[best_acc_idx]) + ', ' + str(all_recalls_std[best_acc_idx]))
print('f1: ' + str(all_f1s[best_acc_idx]) + ', ' + str(all_f1s_std[best_acc_idx]))
print('auc: ' + str(all_aucs[best_acc_idx]) + ', ' + str(all_aucs_std[best_acc_idx]))

accuracy: 0.8100000000000002, 0.13747727084867523
precision: 0.8080952380952381, 0.23405185773422846
recall: 0.8733333333333334, 0.17813852287849852
f1: 0.7962121212121213, 0.18687568236974283
auc: 0.8419444444444446, 0.09770870350557691


This is a stunning result! Although it is not predictable which accuracy you will get with your run because the 200 repeats are created randomly, I obtained a an accuracy of 0.81 with my best repeat For comparison, the paper achieved an accuracy of 0.77 ± 0.13 with an SVM strategy (refer to table 5 in the paper).

However, having a high accuracy on just one repeat does not tell us how well the model performs overall. For this, it is best to take the mean metric scores and standard deviations for all 200 repeats:


In [35]:
print('auc: ' + str(np.mean(all_aucs)) + ', ' + str(np.mean(all_aucs_std)))
print('accuracy: ' + str(np.mean(all_accuracies)) + ', ' + str(np.mean(all_accuracies_std)))
print('f1: ' + str(np.mean(all_f1s)) + ', ' + str(np.mean(all_f1s_std)))
print('precision: ' + str(np.mean(all_precisions)) + ', ' + str(np.mean(all_precisions_std)))
print('recall: ' + str(np.mean(all_recalls)) + ', ' + str(np.mean(all_recalls_std)))

auc: 0.7629810515873016, 0.12542286092697252
accuracy: 0.7514500000000001, 0.1272152270813065
f1: 0.753987681681064, 0.14056347192965324
precision: 0.7279279761904763, 0.18935737052327822
recall: 0.8316603174603174, 0.16566855008491316


# SVM


In [36]:
from sklearn.svm import SVC

In [37]:
# Define some lists to store metric means
all_aucs = []
all_accuracies = []
all_f1s = []
all_precisions = []
all_recalls = []

# Define some lists to store metric standard deviations
all_aucs_std = []
all_accuracies_std = []
all_f1s_std = []
all_precisions_std = []
all_recalls_std = []

# Define the for loop
for i in tqdm(range(200)): # We have 200 repeats
    badsplit = True # initialize badplit to True
    while badsplit: # A loop that runs forever as long as there is a badsplit
        kf = KFold(n_splits=10, shuffle=True) # Create a random K-Fold cross validator with shuffle=True
        splits = list(kf.split(rescaledfeat)) # Split the data and store the splits in a list
        badsplits = [] # Initialize a list to store badsplits
        for j in splits: # A loop that populates badsplits with False or True
            if len(np.unique(Labels[j[1]])) == 2:
                badsplits.append(False) # If there are 2 labels, add False to badsplits
            else:
                badsplits.append(True) # If there aren't 2 labels, add True to badsplts
        badsplit = max(badsplits) # Set badsplit to True if any element in badsplits is True, otherwise False
    # Define some lists to store metrics for this repeat
    aucs = []
    accuracies = []
    f1s = []
    precisions = []
    recalls = []
    # Define a loop to process every one of the 10 CV splits in this repeat
    for s in splits:
        # To use SVM, change the classifier:
        cls = SVC(probability=True) # Define classifier with best params from grid search
        # SVC's syntax is similar to XGBoost, you just need to reshape Labels with ravel or you will get warnings
        cls.fit(rescaledfeat[s[0]], Labels[s[0]].ravel()) # Fit classifier to training data in the split
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1] # Get the predicted probabilities of each class
        pred_labels = np.where(preds_proba > 0.5, 1, 0) # Assign class labels from probabilities using threshold
        # Calculate metrics and append to lists for this repeat
        auc = sklearn.metrics.roc_auc_score(Labels[s[1]], pred_labels)
        aucs.append(auc)
        accuracy = sklearn.metrics.accuracy_score(Labels[s[1]], pred_labels)
        accuracies.append(accuracy)
        f1 = sklearn.metrics.f1_score(Labels[s[1]], pred_labels)
        f1s.append(f1)
        precision = sklearn.metrics.precision_score(Labels[s[1]], pred_labels)
        precisions.append(precision)
        recall = sklearn.metrics.recall_score(Labels[s[1]], pred_labels)
        recalls.append(recall)
    # Once all splits in this repeat are processed, append the means and standard deviations to global lists
    all_aucs.append(np.mean(aucs))
    all_accuracies.append(np.mean(accuracies))
    all_f1s.append(np.mean(f1s))
    all_precisions.append(np.mean(precisions))
    all_recalls.append(np.mean(recalls))
    all_aucs_std.append(np.std(aucs))
    all_accuracies_std.append(np.std(accuracies))
    all_f1s_std.append(np.std(f1s))
    all_precisions_std.append(np.std(precisions))
    all_recalls_std.append(np.std(recalls))

  0%|          | 0/200 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  0%|          | 1/200 [00:00<00:28,  7.06it/s]/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
  1%|          | 2/200 [00:00<00:27,  7.25it/s]/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

In [38]:
best_acc_idx = np.argmax(all_accuracies)
print('accuracy: ' + str(all_accuracies[best_acc_idx]) + ', ' + str(all_accuracies_std[best_acc_idx]))
print('precision: ' + str(all_precisions[best_acc_idx]) + ', ' + str(all_precisions_std[best_acc_idx]))
print('recall: ' + str(all_recalls[best_acc_idx]) + ', ' + str(all_recalls_std[best_acc_idx]))
print('f1: ' + str(all_f1s[best_acc_idx]) + ', ' + str(all_f1s_std[best_acc_idx]))
print('auc: ' + str(all_aucs[best_acc_idx]) + ', ' + str(all_aucs_std[best_acc_idx]))

accuracy: 0.53, 0.20024984394500786
precision: 0.4254761904761904, 0.2924717750980852
recall: 0.45666666666666667, 0.34730710073682947
f1: 0.4226767676767677, 0.2896088762344064
auc: 0.5648809523809524, 0.17967930142491154


In [39]:
print('auc: ' + str(np.mean(all_aucs)) + ', ' + str(np.mean(all_aucs_std)))
print('accuracy: ' + str(np.mean(all_accuracies)) + ', ' + str(np.mean(all_accuracies_std)))
print('f1: ' + str(np.mean(all_f1s)) + ', ' + str(np.mean(all_f1s_std)))
print('precision: ' + str(np.mean(all_precisions)) + ', ' + str(np.mean(all_precisions_std)))
print('recall: ' + str(np.mean(all_recalls)) + ', ' + str(np.mean(all_recalls_std)))

auc: 0.43082251984126985, 0.13058662141450866
accuracy: 0.3833, 0.12868666404520548
f1: 0.30749178044178044, 0.1921938161034327
precision: 0.34172341269841267, 0.23724896302180482
recall: 0.36290079365079364, 0.2861775416513885


# Better hyperparameter search

You can find hyperparameters faster using Optuna:


In [40]:
!pip install optuna==3.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 26.1 MB/s eta 0:00:00


In [41]:
import optuna

In [42]:
splits = list(kf.split(rescaledfeat, Labels))

In [43]:
def objective(trial):
    param = {
        "device": "cuda",
        "verbosity": 0,
        "objective": "binary:logistic",
        "random_state": 42,
        # use exact for small dataset.
        "tree_method": "hist",
        "max_bin": 32,
        "n_estimators": trial.suggest_int("n_estimators", 50, 350, log=True),
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1.0, 10, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 10, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0, log=True),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 0.1, log=True),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 1, 12, log=True)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 1, 10, log=True)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    mymetrics = []
    for s in splits:
        cls = xgb.XGBClassifier(**param)
        cls.fit(rescaledfeat[s[0]], Labels[s[0]])
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1]
        metric = sklearn.metrics.roc_auc_score(Labels[s[1]], preds_proba)
        mymetrics.append(metric)
    
    return np.mean(mymetrics)

In [44]:
sampler = optuna.samplers.TPESampler(multivariate=True, group=True, constant_liar=True, seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=5000, timeout=120)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print(study.best_params)

/opt/conda/lib/python3.11/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/optuna/samplers/_tpe/sampler.py:330: ExperimentalWarning: ``group`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/optuna/samplers/_tpe/sampler.py:338: ExperimentalWarning: ``constant_liar`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-09-27 11:28:59,126] A new study created in memory with name: no-name-cf9e683b-eedb-46c9-a200-69113b5942b1
[I 2024-09-27 11:28:59,420] Trial 0 finished with value: 0.7218214285714286 and parameters: {'n_estimators': 103, 'booster': 'gbtree', 'lambda': 3.968793330444372, 'alpha': 2.5361081166471375e-07, 'subsample': 0.25707833957860277, 'colsample_bytree': 0.011430983876313222

Number of finished trials:  17
Best trial:
  Value: 0.8182023809523811
  Params: 
    n_estimators: 75
    booster: dart
    lambda: 2.0148477884158655
    alpha: 0.00052821153945323
    subsample: 0.40081743753083116
    colsample_bytree: 0.019553708662745254
    max_depth: 4
    min_child_weight: 1
    eta: 2.1734877073417355e-06
    gamma: 8.528933855762793e-06
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: tree
    rate_drop: 0.0007250347382396634
    skip_drop: 2.3130924416844053e-07
{'n_estimators': 75, 'booster': 'dart', 'lambda': 2.0148477884158655, 'alpha': 0.00052821153945323, 'subsample': 0.40081743753083116, 'colsample_bytree': 0.019553708662745254, 'max_depth': 4, 'min_child_weight': 1, 'eta': 2.1734877073417355e-06, 'gamma': 8.528933855762793e-06, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.0007250347382396634, 'skip_drop': 2.3130924416844053e-07}


In [45]:
from copy import deepcopy

In [46]:
default_params = {"device": "cuda",
                  "verbosity": 0,
                  "objective": "binary:logistic",
                  "random_state": 42,
                  # use exact for small dataset.
                  "tree_method": "hist",
                  "max_bin": 32}

all_params = deepcopy(study.best_params)
all_params.update(default_params)

print(all_params)

all_aucs = []
all_accuracies = []
all_f1s = []
all_precisions = []
all_recalls = []

for i in tqdm(range(200)):
    badsplit = True
    while badsplit:
        kf = KFold(n_splits=10, shuffle=True)
        splits = list(kf.split(rescaledfeat))
        badsplits = []
        for j in splits:
            if len(np.unique(Labels[j[1]])) == 2:
                badsplits.append(False)
            else:
                badsplits.append(True)
        badsplit = max(badsplits)
    aucs = []
    accuracies = []
    f1s = []
    precisions = []
    recalls = []
    for s in splits:
        cls = xgb.XGBClassifier(**all_params)
        cls.fit(rescaledfeat[s[0]], Labels[s[0]])
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1]
        pred_labels = np.where(preds_proba > 0.5, 1, 0)
        auc = sklearn.metrics.roc_auc_score(Labels[s[1]], pred_labels)
        aucs.append(auc)
        accuracy = sklearn.metrics.accuracy_score(Labels[s[1]], pred_labels)
        accuracies.append(accuracy)
        f1 = sklearn.metrics.f1_score(Labels[s[1]], pred_labels)
        f1s.append(f1)
        precision = sklearn.metrics.precision_score(Labels[s[1]], pred_labels)
        precisions.append(precision)
        recall = sklearn.metrics.recall_score(Labels[s[1]], pred_labels)
        recalls.append(recall)
    all_aucs.append(np.mean(aucs))
    all_accuracies.append(np.mean(accuracies))
    all_f1s.append(np.mean(f1s))
    all_precisions.append(np.mean(precisions))
    all_recalls.append(np.mean(recalls))

{'n_estimators': 75, 'booster': 'dart', 'lambda': 2.0148477884158655, 'alpha': 0.00052821153945323, 'subsample': 0.40081743753083116, 'colsample_bytree': 0.019553708662745254, 'max_depth': 4, 'min_child_weight': 1, 'eta': 2.1734877073417355e-06, 'gamma': 8.528933855762793e-06, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.0007250347382396634, 'skip_drop': 2.3130924416844053e-07, 'device': 'cuda', 'verbosity': 0, 'objective': 'binary:logistic', 'random_state': 42, 'tree_method': 'hist', 'max_bin': 32}


  0%|          | 0/200 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-pac

In [47]:
best_acc_idx = np.argmax(all_accuracies)
print('accuracy: ' + str(all_accuracies[best_acc_idx]) + ', ' + str(all_accuracies_std[best_acc_idx]))
print('precision: ' + str(all_precisions[best_acc_idx]) + ', ' + str(all_precisions_std[best_acc_idx]))
print('recall: ' + str(all_recalls[best_acc_idx]) + ', ' + str(all_recalls_std[best_acc_idx]))
print('f1: ' + str(all_f1s[best_acc_idx]) + ', ' + str(all_f1s_std[best_acc_idx]))
print('auc: ' + str(all_aucs[best_acc_idx]) + ', ' + str(all_aucs_std[best_acc_idx]))

accuracy: 0.58, 0.1345362404707371
precision: 0.4705555555555555, 0.33181149198475923
recall: 0.74, 0.28019915172221765
f1: 0.5631968031968031, 0.22927648368569636
auc: 0.62, 0.1838804452370876


In [48]:
print('auc: ' + str(np.mean(all_aucs)) + ', ' + str(np.mean(all_aucs_std)))
print('accuracy: ' + str(np.mean(all_accuracies)) + ', ' + str(np.mean(all_accuracies_std)))
print('f1: ' + str(np.mean(all_f1s)) + ', ' + str(np.mean(all_f1s_std)))
print('precision: ' + str(np.mean(all_precisions)) + ', ' + str(np.mean(all_precisions_std)))
print('recall: ' + str(np.mean(all_recalls)) + ', ' + str(np.mean(all_recalls_std)))

auc: 0.5482, 0.13058662141450866
accuracy: 0.4307000000000001, 0.12868666404520548
f1: 0.3799289932289932, 0.1921938161034327
precision: 0.29134781746031746, 0.23724896302180482
recall: 0.6086, 0.2861775416513885
